<a href="https://colab.research.google.com/github/whitehatjr1001/Counterfit-note-detection/blob/main/Fake_currency_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import numpy as np 
from keras.layers import Dense,Activation,Flatten,Dropout,Conv2D
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import load_img
from keras.models import Sequential,Model,load_model
import  os 
import shutil

In [2]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
height = 300
width = 300
train_path = '/content/drive/MyDrive/Dataset/Training'
valid_path = '/content/drive/MyDrive/Dataset/Validation'

In [4]:
os.listdir(train_path)

['Fake', 'Real']

In [4]:
class_list =['Fake', 'Real']

for item in class_list:
    
    # We are creating temporary directories here because we delete these directories later
    # create a base dir
    
    aug_dir = 'aug_dir'
    os.mkdir(aug_dir)
    # create a dir within the base dir to store images of the same class
    img_dir = os.path.join(aug_dir, 'img_dir')
    os.mkdir(img_dir)

    # Choose a class
    img_class = item

    # list all images in that directory
    img_list = os.listdir('/content/drive/MyDrive/Dataset/Training/' + img_class)

    # Copy images from the class train dir to the img_dir e.g. class 'mel'
    for fname in img_list:
            # source path to image
            src = os.path.join('/content/drive/MyDrive/Dataset/Training/' + img_class, fname)
            # destination path to image
            dst = os.path.join(img_dir, fname)
            # copy the image from the source to the destination
            shutil.copyfile(src, dst)


    # point to a dir containing the images and not to the images themselves
    path = aug_dir
    save_path = '/content/drive/MyDrive/Dataset/Training/' + img_class

    # Create a data generator
    datagen = ImageDataGenerator(
        rotation_range=180,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
        vertical_flip=True,
        brightness_range=(0.9,1.1),
        fill_mode='nearest')

    batch_size = 50

    aug_datagen = datagen.flow_from_directory(path,
                                           save_to_dir=save_path,
                                           save_format='jpg',
                                                    target_size=(224,224),
                                                    batch_size=batch_size)



    # Generate the augmented images and add them to the training folders
    
    ###########
    
    num_aug_images_wanted = 1000 # total number of images we want to have in each class
    
    ###########
    
    num_files = len(os.listdir(img_dir))
    num_batches = int(np.ceil((num_aug_images_wanted-num_files)/batch_size))

    # run the generator and create about 6000 augmented images
    for i in range(0,num_batches):

        imgs, labels = next(aug_datagen)
        
    # delete temporary directory with the raw image files
    shutil.rmtree('aug_dir')
     


Found 1278 images belonging to 1 classes.
Found 483 images belonging to 1 classes.


In [5]:
import tensorflow

In [10]:
train_batch_size = 8
val_batch_size = 8


In [11]:
datagen = ImageDataGenerator(
    preprocessing_function= \
    tensorflow.keras.applications.mobilenet.preprocess_input)

train_batches = datagen.flow_from_directory(train_path,
                                            target_size=(height,width),
                                            batch_size=train_batch_size)

valid_batches = datagen.flow_from_directory(valid_path,
                                            target_size=(height,width),
                                            batch_size=val_batch_size)

# Note: shuffle=False causes the test dataset to not be shuffled
test_batches = datagen.flow_from_directory(valid_path,
                                            target_size=(height,height),
                                            batch_size=1,
                                            shuffle=False)

Found 2294 images belonging to 2 classes.
Found 113 images belonging to 2 classes.
Found 113 images belonging to 2 classes.


1761

In [12]:
num_train_samples = train_batches.n
num_val_samples = valid_batches.n


train_steps = np.ceil(num_train_samples / train_batch_size)
val_steps = np.ceil(num_val_samples / val_batch_size)

In [27]:
import tensorflow as tf 
     

In [28]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(height, width, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    # flatten and dense layers
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [29]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 298, 298, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 149, 149, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 147, 147, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 73, 73, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 71, 71, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 35, 35, 128)      0

In [30]:
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam, SGD
import math

In [31]:
early_stopping = EarlyStopping(monitor='val_loss',patience=2)

checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath = 'model.hdf5',
                                                  monitor = 'val_accuracy', 
                                                  verbose = 1, 
                                                  save_best_only = True,
                                                  save_weights_only = True,
                                                  mode = 'max')

def step_decay(epoch):
    initial_lrate = 0.001
    drop = 0.1
    epochs_drop = 10.0
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(step_decay, verbose=1)

In [32]:
callbacks = [ checkpointer, lr_scheduler]

In [33]:

history = model.fit_generator(train_batches, steps_per_epoch=train_steps,
                              validation_data=valid_batches,
                              epochs=50,
                              callbacks=callbacks
                             )


Epoch 1: LearningRateScheduler setting learning rate to 0.001.
Epoch 1/50


<ipython-input-33-39c2901732df>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_batches, steps_per_epoch=train_steps,


287/287 [==============================] - ETA: 0s - loss: 0.8249 - accuracy: 0.5000
Epoch 1: val_accuracy improved from -inf to 0.50000, saving model to model.hdf5
287/287 [==============================] - 58s 186ms/step - loss: 0.8249 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.001.
Epoch 2/50
198/287 [===================>..........] - ETA: 5s - loss: 0.6931 - accuracy: 0.5000

KeyboardInterrupt: ignored

In [13]:
from tensorflow.keras.applications.resnet50 import ResNet50,preprocess_input

base_model=ResNet50(weights='imagenet',include_top=False,input_shape=(height,width,3))

94765736/94765736 [==============================] - 7s 0us/step


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [14]:
def build_finetune_model(base_model,dropout,fc_layers,num_classes):
    for layer in base_model.layers:
        layer.trainable=False

    x=base_model.output
    x=Flatten()(x)
    for fc in fc_layers:
        x=Dense(fc,activation='relu')(x)
        x=Dropout(dropout)(x)
    
    predictions=Dense(num_classes,activation='softmax')(x)

    finetune_model=Model(inputs=base_model.input,outputs=predictions) 
    
    return finetune_model

class_list=['Real','Fake'] #the labels of our data
FC_Layers=[1024,1024]
dropout=0.5

finetune_model=build_finetune_model(base_model,dropout=dropout,fc_layers=FC_Layers,num_classes=len(class_list))

In [15]:
num_epochs=100
num_train_images=train_batches.n
batch_size=8

In [18]:
from keras import optimizers
from keras.callbacks import ModelCheckpoint,EarlyStopping

In [34]:

checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath = 'model.hdf5',
                                                  monitor = 'val_accuracy', 
                                                  verbose = 1, 
                                                  save_best_only = True,
                                                  save_weights_only = True,
                                                  mode = 'max')
early_stopping = EarlyStopping(monitor='val_loss',patience=40)


In [ ]:
#compile the model before using
finetune_model.compile(loss="categorical_crossentropy",optimizer=optimizers.SGD(lr=0.000001,momentum=0.9),metrics=['accuracy'])

#train the model
finetune_model.fit_generator(generator=train_batches,steps_per_epoch=num_train_images//batch_size,epochs=num_epochs,validation_data=valid_batches,callbacks=[checkpointer,early_stopping])

#save the model
finetune_model.save_weights("Final_model.h5")

Epoch 1/100


<ipython-input-36-b7844f967135>:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  finetune_model.fit_generator(generator=train_batches,steps_per_epoch=num_train_images//batch_size,epochs=num_epochs,validation_data=valid_batches,callbacks=[checkpointer,early_stopping])


286/286 [==============================] - ETA: 0s - loss: 0.6391 - accuracy: 0.6317
Epoch 1: val_accuracy improved from -inf to 0.25000, saving model to model.hdf5
286/286 [==============================] - 34s 108ms/step - loss: 0.6391 - accuracy: 0.6317 - val_loss: 0.6749 - val_accuracy: 0.2500
Epoch 2/100
286/286 [==============================] - ETA: 0s - loss: 0.6230 - accuracy: 0.6382
Epoch 2: val_accuracy did not improve from 0.25000
286/286 [==============================] - 26s 89ms/step - loss: 0.6230 - accuracy: 0.6382 - val_loss: 0.8164 - val_accuracy: 0.1250
Epoch 3/100
286/286 [==============================] - ETA: 0s - loss: 0.6201 - accuracy: 0.6592
Epoch 3: val_accuracy improved from 0.25000 to 1.00000, saving model to model.hdf5
286/286 [==============================] - 30s 106ms/step - loss: 0.6201 - accuracy: 0.6592 - val_loss: 0.3865 - val_accuracy: 1.0000
Epoch 4/100
286/286 [==============================] - ETA: 0s - loss: 0.6147 - accuracy: 0.6645
Epoch 4: 